In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model

data_4 = pd.read_excel('dream4_data.xlsx')
data_3 = pd.read_excel('dream3_data.xlsx')



In [2]:
print len(data_4)
data_4.dropna(inplace=True)
print len(data_4)


data_3 = data_3[['DP', 'FS', 'Label', 'RAW_MQ', 'TLOD_FWD', 'TLOD_REV', 'BaseQRankSum', 'ClippingRankSum', 'MQRankSum', 'ReadPosRankSum']]

print len(data_3)
data_3.dropna(inplace=True)
print len(data_3)

def normalize_value(df, column, mean, std):
    name = column + '_norm'
    df[name] = (df[column] - mean) / std
    return df
    
def build_summary(df, columns):
    """
    df is a dataframe
    columns is a list of columns get summary data
    returns a dict with column as key - value is (median, std)
    """
    summary_dict = dict()
    for name in columns:
        mean = np.mean(df[name])
        std = np.std(df[name])
        summary_dict[name] = (mean, std)
    return summary_dict

def normalize_cols(df, columns):
    col_dict = build_summary(df, columns)
    for col in columns:
        stats = col_dict[col]
        normalize_value(df, col, stats[0], stats[1])
    return df
cols = ['DP', 'FS', 'RAW_MQ', 'TLOD_FWD', 'TLOD_REV', 'BaseQRankSum', 'ClippingRankSum', 'MQRankSum', 'ReadPosRankSum']
normalize_cols(data_3, cols)
normalize_cols(data_4, cols)


print data_3['DP_norm'][0:10]

43923
43688
56791
56372
0   -0.337144
1    0.029098
2   -0.091133
3   -0.248358
4   -0.128127
5    1.686433
6   -0.118879
7   -0.316797
8   -0.313098
9   -0.296450
Name: DP_norm, dtype: float64


In [3]:
Y_dream3 = data_3['Label']
Y_dream4 = data_4['Label']

del data_3['Label']
del data_4['Label']

data_test = data_4
Y_test = Y_dream4
data = data_3
Y = Y_dream3

X_base = np.random.randint(2, size=len(data)).reshape(-1, 1)
X_base_test = np.random.randint(2, size=len(data_test)).reshape(-1, 1)

X_1 = data.as_matrix(columns=['DP', 'FS'])
X_1_test = data_test.as_matrix(columns=['DP', 'FS'])

X_2 = data.as_matrix(columns=['DP', 'FS', 'RAW_MQ'])
X_2_test = data_test.as_matrix(columns=['DP', 'FS', 'RAW_MQ'])

X_3 = data.as_matrix(columns=['DP', 'FS', 'RAW_MQ','TLOD_FWD', 'TLOD_REV'])
X_3_test = data_test.as_matrix(columns=['DP', 'FS', 'RAW_MQ', 'TLOD_FWD', 'TLOD_REV'])

X_4 = data.as_matrix(columns=['DP', 'FS','TLOD_FWD', 'TLOD_REV'])
X_4_test = data_test.as_matrix(columns=['DP', 'FS', 'TLOD_FWD', 'TLOD_REV'])

X_5 = data.as_matrix(columns=['TLOD_FWD', 'TLOD_REV'])
X_5_test = data_test.as_matrix(columns=['TLOD_FWD', 'TLOD_REV'])

X_6 = data.as_matrix(columns=['DP_norm', 'FS_norm', 'RAW_MQ_norm','TLOD_FWD_norm', 'TLOD_REV_norm'])
X_6_test = data_test.as_matrix(columns=['DP_norm', 'FS_norm', 'RAW_MQ_norm', 'TLOD_FWD_norm', 'TLOD_REV_norm'])

X_7 = data.as_matrix(columns=['DP_norm', 'FS_norm'])
X_7_test = data_test.as_matrix(columns=['DP_norm', 'FS_norm'])

X_8 = data.as_matrix(columns=['DP_norm', 'FS_norm', 'RAW_MQ_norm','TLOD_FWD_norm', 'TLOD_REV_norm', 'BaseQRankSum_norm', 'ClippingRankSum_norm', 'MQRankSum_norm', 'ReadPosRankSum_norm'])
X_8_test = data_test.as_matrix(columns=['DP_norm', 'FS_norm', 'RAW_MQ_norm', 'TLOD_FWD_norm', 'TLOD_REV_norm', 'BaseQRankSum_norm', 'ClippingRankSum_norm', 'MQRankSum_norm', 'ReadPosRankSum_norm'])


#X_1 = data.as_matrix(columns=None)
#X_3 = data.as_matrix(columns=['BaseQRankSum', 'ClippingRankSum', 'DP', 'FS', 'MQRankSum', 'N_ART_LOD', 'RAW_MQ', 
#                              'ReadPosRankSum', 'SOR','TLOD', 'TLOD_FWD', 'TLOD_REV'])


In [4]:

logreg = linear_model.LogisticRegression(C=1)
logreg.fit(X_base, Y)
print logreg.score(X_base_test, Y_test)

logreg.fit(X_1, Y)
print logreg.score(X_1_test, Y_test)

logreg.fit(X_2, Y)
print logreg.score(X_2_test, Y_test)

logreg.fit(X_3, Y)
print logreg.score(X_3_test, Y_test)

logreg.fit(X_4, Y)
print logreg.score(X_4_test, Y_test)

logreg.fit(X_5, Y)
print logreg.score(X_5_test, Y_test)

logreg.fit(X_6, Y)
print logreg.score(X_6_test, Y_test)

logreg.fit(X_7, Y)
print logreg.score(X_7_test, Y_test)

logreg.fit(X_8, Y)
print logreg.score(X_8_test, Y_test)


0.48928767625
0.476194836111
0.48928767625
0.489058780443
0.623100164805
0.473539644754
0.600210584142
0.48928767625
0.576977659769


In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

scores = cross_val_score(logreg, X_8, Y, cv=7)
print scores
print scores.mean(), scores.std()

[ 0.89880805  0.88377002  0.88091395  0.89693282  0.88699863  0.89780206
  0.88923383]
0.890637050823 0.00670438244385


In [6]:
##### SVM Classifier



In [7]:
### RBF KERNEL
svc = SVC(kernel='rbf')
svc.fit(X_1, Y)
print svc.score(X_1_test, Y_test)

svc.fit(X_2, Y)
print svc.score(X_2_test, Y_test)

svc.fit(X_3, Y)
print svc.score(X_3_test, Y_test)

svc.fit(X_4, Y)
print svc.score(X_4_test, Y_test)

svc.fit(X_5, Y)
print svc.score(X_5_test, Y_test)

svc.fit(X_6, Y)
print svc.score(X_6_test, Y_test)

svc.fit(X_7, Y)
print svc.score(X_7_test, Y_test)

svc.fit(X_8, Y)
print svc.score(X_8_test, Y_test)


0.520165720564
0.68268174327
0.615088811573
0.693050723311
0.612158945248
0.647569126534
0.48928767625
0.666567478484


In [8]:
scores = cross_val_score(svc, X_8, Y, cv=7)
print scores
print scores.mean(), scores.std()

[ 0.9364291   0.92238917  0.91779461  0.92214082  0.91891221  0.9359245
  0.92971563]
0.926186577689 0.00723405402536
